# Tools Agents

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

In [3]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [4]:
api_wrapper_wikipedia = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)

In [5]:
wikipedia=WikipediaQueryRun(api_wrapper=api_wrapper_wikipedia)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

In [6]:
wikipedia.name

'wikipedia'

In [7]:
arxiv.name

'arxiv'

In [8]:
tools=[wikipedia,arxiv]

In [9]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/iceyisaak/anaconda3/envs/langchain-chatbot-search-engine-tools-agents/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

---

## Custom Tools - RAG Tools

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

/home/iceyisaak/anaconda3/envs/langchain-chatbot-search-engine-tools-agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
webbase_loader = WebBaseLoader("https://docs.langchain.com/")
docs=webbase_loader.load()
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks=splitter.split_documents(docs)
openai_embeddings=OpenAIEmbeddings()
vector_store=FAISS.from_documents(chunks,openai_embeddings)
retriever=vector_store.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e8300e716d0>, search_kwargs={})

### Retriever Tool

In [12]:
from langchain_core.tools.retriever import create_retriever_tool

In [13]:
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any info about Langsmith")

In [14]:
retriever_tool

StructuredTool(name='langsmith-search', description='Search any info about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x7e830096e700>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x7e830096e980>)

---

In [15]:
tools=[wikipedia,arxiv,retriever_tool]

In [16]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/iceyisaak/anaconda3/envs/langchain-chatbot-search-engine-tools-agents/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-search', description='Search any info about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x7e830096e700>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x7e830096e980>)]

---

## Agent Executor

In [17]:
from langchain_groq import ChatGroq

In [18]:
llm=ChatGroq(model='llama-3.1-8b-instant')

#### Prompt Hub

In [19]:
from langchain_classic import hub

In [20]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [21]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [22]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agents

In [26]:
from langchain_classic.agents import create_openai_tools_agent

In [27]:
agent=create_openai_tools_agent(llm,tools,prompt)

In [28]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

### Query using Agent Executor

In [29]:
from langchain_classic.agents import AgentExecutor

In [31]:
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [32]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [33]:
agent_executor.invoke({
    "input":"Tell me about Langsmith"
})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'Langsmith'}`


for long-running tasks.Learn moreAgent Builder (Beta)Turn natural-language ideas into production agents with persistent memory and self-updating capabilities.Learn moreLangSmith meets the highest standards of data security and privacy with HIPAA, SOC 2 Type 2, and GDPR compliance. For more information, see the Trust Center.Get startedBuild your first agent with LangChainGet startedSign up for LangSmithTry LangSmithBuild an advanced agent with LangGraphGet startedEnroll in LangChain AcademyGet startedAdditional resourcesCommunity forumAsk questions, share solutions, and discuss best practices.JoinCommunity SlackConnect with other builders and get quick help.JoinSupport portalSubmit tickets and track support requests.VisitLangSmith statusReal-time status of LangSmith services and APIs.View

moreLangGraph (TypeScript)Control every step of your custom agent with low-level orchestration, me

{'input': 'Tell me about Langsmith',
 'output': 'It seems like the provided information is only available on the Langsmith website and is not a suitable answer. Let me try to provide a more general answer.\n\nLangsmith is a platform that helps AI teams use live production data for continuous testing and improvement. It provides features such as observability, evaluation, prompt engineering, and deployment to help teams build and maintain reliable agents.\n\nIf you are looking for more information, I can try to provide a more general answer or try to find more information on the topic.'}

In [34]:
agent_executor.invoke({
    "input":"What is machine learning?"
})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'machine learning algorithms'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `wikipedia` with `{'query': 'machine learning applications'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit

{'input': 'What is machine learning?',
 'output': 'It seems like the wikipedia page for machine learning has no information on its definition.'}

In [36]:
agent_executor.invoke({
    "input":"What is the paper 1706.03762 about?"
})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': '1706.03762'}`
responded: It seems the function call returned a partial response. To get the full response let's try again.



Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `arxiv` with `{'query': 'Attention Is All You Need'}`


Published: 2021-05-06
Title: Do You Even Need Attention? A Stack of Feed-Forward Layers Does Surprisingly Well on ImageNet
Authors: Luke Melas-Kyriazi
Summary: The strong performa

{'input': 'What is the paper 1706.03762 about?',
 'output': 'The paper 1706.03762, titled "Attention Is All You Need," presents a new approach to sequence transduction models that relies solely on self-attention mechanisms, without the need for recurrent neural networks (RNNs) or convolutional neural networks (CNNs). The paper introduces the Transformer model, which consists of an encoder and a decoder, and demonstrates its effectiveness on a variety of tasks, including machine translation and text summarization. The authors show that the Transformer model outperforms traditional sequence-to-sequence models, such as those based on LSTMs, and achieves state-of-the-art results on several benchmark datasets.'}

---